# RunViso Environment Setup (Vast.ai Base Image)

This notebook installs project dependencies into the default `/venv/main/` Python environment, clones the VisoMaster repository into `/workspace/VisoMaster`, and runs the model download script.

**Environment:**
*   Base Image: `vastai/base-image`
*   Workspace: `/workspace`
*   Python Env: `/venv/main/`
*   Logs: Check `/var/log/supervisor/` and `/var/log/portal/` via terminal or Instance Portal UI. Check `/workspace/entrypoint.log` for full startup log (IF entrypoint logging is enabled).

## Section 0: Setup Widgets and Helper Function

Initializes widgets for status display and defines a helper function to run shell commands via Python's `subprocess`, using the default Python environment.

In [ ]:
import subprocess
import shlex
import ipywidgets as widgets
from IPython.display import display, HTML
import os
import sys
import shutil # For rmtree and which
import time # For potential real-time output simulation

# --- Configuration ---
PYTHON_ENV_PATH = "/venv/main"
PYTHON_EXE = os.path.join(PYTHON_ENV_PATH, "bin/python")
PIP_EXE = os.path.join(PYTHON_ENV_PATH, "bin/pip")
REPO_URL = "https://github.com/remphan1618/VisoMaster.git"
REPO_DIR_NAME = "VisoMaster"
WORKSPACE_DIR = "/workspace" # Standard vast.ai workspace
REPO_CLONE_PATH = os.path.join(WORKSPACE_DIR, REPO_DIR_NAME)
# Log files for notebook steps will be placed in workspace for easy access via Jupyter
NOTEBOOK_LOG_DIR = os.path.join(WORKSPACE_DIR, "setup_logs") 

# Ensure Python/Pip executables exist
if not os.path.exists(PYTHON_EXE):
    print(f"ERROR: Python executable not found at {PYTHON_EXE}", file=sys.stderr)
if not os.path.exists(PIP_EXE):
    print(f"ERROR: Pip executable not found at {PIP_EXE}", file=sys.stderr)

# Ensure notebook log directory exists
os.makedirs(NOTEBOOK_LOG_DIR, exist_ok=True)

# --- Widgets for Status Feedback ---
status_output = widgets.Output() # This will capture print statements
overall_progress = widgets.FloatProgress(value=0.0, min=0.0, max=1.0, description='Overall:', bar_style='info', style={'description_width': 'initial'})
step_description = widgets.Label(value="Starting setup...")

display(step_description, overall_progress, status_output)

# --- Helper Function to Run Commands (Modified for direct output) ---
def run_command(command_list, step_name, log_file_name, env_vars=None, cwd=None):
    """Runs a command list using subprocess, logs output, updates status, and prints output to cell."""
    log_path = os.path.join(NOTEBOOK_LOG_DIR, log_file_name)
    status_html = f"<p><strong>Running:</strong> {step_name}... Log: <a href='{os.path.join('setup_logs', log_file_name)}' target='_blank'>{log_path}</a></p>"
    display(HTML(status_html))
    
    # Combine current environment with any extra vars
    run_env = os.environ.copy()
    if env_vars:
        run_env.update(env_vars)
        
    success = False
    full_output = ""
    try:
        # Use Popen to potentially stream output later if needed, but capture for now
        process = subprocess.Popen(
            command_list, 
            stdout=subprocess.PIPE, 
            stderr=subprocess.STDOUT, # Redirect stderr to stdout
            text=True, 
            encoding='utf-8', # Ensure consistent encoding
            errors='replace', # Handle potential decoding errors
            bufsize=1, # Line buffered
            env=run_env, 
            cwd=cwd
        )
        
        # Read and print output line by line, also save to log and full_output string
        with open(log_path, 'w', encoding='utf-8') as f_log:
            with status_output: # Direct print statements to the widget's output area
                print(f"--- Output for {step_name} --- ")
                while True:
                    line = process.stdout.readline()
                    if not line:
                        break
                    line_stripped = line.rstrip() # Remove trailing newline for cleaner print
                    print(line_stripped) # Print to notebook cell output
                    f_log.write(line) # Write original line (with newline) to log
                    f_log.flush() # Ensure it's written immediately
                    full_output += line
                print(f"--- End Output for {step_name} --- ")
            
        process.wait() # Wait for the process to fully terminate

        if process.returncode == 0:
            status_html = f"<p><strong>Status:</strong> {step_name} <font color='green'>Success.</font> Log: <a href='{os.path.join('setup_logs', log_file_name)}' target='_blank'>{log_path}</a></p>"
            success = True
        else:
            status_html = f"<p><strong>Status:</strong> {step_name} <font color='red'>Failed (Code: {process.returncode}).</font> Log: <a href='{os.path.join('setup_logs', log_file_name)}' target='_blank'>{log_path}</a></p>"
            with status_output:
                 print(f"ERROR: {step_name} failed with code {process.returncode}. Check log file.", file=sys.stderr)
            success = False
            
    except Exception as e:
        error_message = f"Exception during '{step_name}': {e}"
        status_html = f"<p><strong>Status:</strong> {step_name} <font color='red'>Exception occurred.</font> Details: {error_message}. Log: <a href='{os.path.join('setup_logs', log_file_name)}' target='_blank'>{log_path}</a></p>"
        with status_output:
            print(f"EXCEPTION during {step_name}: {error_message}", file=sys.stderr)
        # Write exception to log file as well
        try:
            with open(log_path, 'a', encoding='utf-8') as f_log:
                f_log.write(f"\n--- Exception Occurred ---\n{error_message}\n")
        except Exception as log_e:
             with status_output:
                 print(f"Failed to write exception to log: {log_e}\n", file=sys.stderr)
        success = False
    finally:
        # Display the final HTML status update
        display(HTML(status_html))
        
    return success

with status_output:
    print(f"Helper function and widgets initialized. Workspace: {WORKSPACE_DIR}, Python: {PYTHON_EXE}")

## Section 1: Clone VisoMaster Repository

Clones the specified Git repository into `/workspace/VisoMaster`.

In [ ]:
step_description.value = "Cloning VisoMaster Repository..."
overall_progress.value = 0.1
status_output.clear_output(wait=True) # Clear previous output before running

# Find git executable
git_exe = shutil.which('git')
if not git_exe:
    with status_output:
        print("Error: 'git' command not found in PATH.\n", file=sys.stderr)
    overall_progress.bar_style = 'danger'
    step_description.value = "Error: Git not found."
else:
    # Remove existing repo directory if it exists to ensure clean clone
    if os.path.exists(REPO_CLONE_PATH):
        with status_output:
            print(f"Removing existing directory: {REPO_CLONE_PATH}\n")
        try:
            shutil.rmtree(REPO_CLONE_PATH)
        except Exception as e:
            with status_output:
                print(f"Error removing directory: {e}\n", file=sys.stderr)
            # Proceeding anyway, git clone might handle it or fail

    clone_command = [git_exe, 'clone', REPO_URL, REPO_CLONE_PATH]
    success = run_command(clone_command, "Git Clone", "git_clone.log", cwd=WORKSPACE_DIR)

    if not success:
        step_description.value = "Error during Git Clone. Check logs and output above."
        overall_progress.bar_style = 'danger'
    else:
        step_description.value = "Git Clone Complete."
        overall_progress.value = 0.3

## Section 2: Install Python Dependencies (from requirements.txt)

Installs packages listed in `VisoMaster/requirements.txt` into the `/venv/main/` environment using pip.

In [ ]:
step_description.value = f"Installing Python deps from requirements.txt..."
overall_progress.value = 0.4
status_output.clear_output(wait=True)

requirements_path = os.path.join(REPO_CLONE_PATH, 'requirements.txt')

if not os.path.exists(requirements_path):
    err_msg = f"Error: requirements.txt not found at {requirements_path}. Check Git Clone step."
    with status_output:
        print(err_msg, file=sys.stderr)
    display(HTML(f"<p><font color='red'>{err_msg}</font></p>"))
    step_description.value = "Error: requirements.txt not found."
    overall_progress.bar_style = 'danger'
elif not os.path.exists(PIP_EXE):
    err_msg = f"Error: pip executable not found at {PIP_EXE}."
    with status_output:
        print(err_msg, file=sys.stderr)
    display(HTML(f"<p><font color='red'>{err_msg}</font></p>"))
    step_description.value = "Error: pip not found."
    overall_progress.bar_style = 'danger'
else:
    pip_install_cmd = [PIP_EXE, 'install', '-r', requirements_path]
    with status_output:
        print(f"Using command: {' '.join(pip_install_cmd)}\n")
    success = run_command(pip_install_cmd, "Pip Install Requirements", "pip_install.log")

    if not success:
        step_description.value = "Error during Pip Install. Check logs and output above."
        overall_progress.bar_style = 'danger'
    else:
        step_description.value = "Pip Install Complete."
        overall_progress.value = 0.7

## Section 3: Run Model Download Script

Executes the `download_models.py` script from the VisoMaster repository using the `/venv/main/bin/python` interpreter.

In [ ]:
step_description.value = f"Running download_models.py script..."
overall_progress.value = 0.8
status_output.clear_output(wait=True)

script_path = os.path.join(REPO_CLONE_PATH, 'download_models.py')

if not os.path.exists(script_path):
    err_msg = f"Error: download_models.py not found at {script_path}. Check Git Clone step."
    with status_output:
        print(err_msg, file=sys.stderr)
    display(HTML(f"<p><font color='red'>{err_msg}</font></p>"))
    step_description.value = "Error: download_models.py not found."
    overall_progress.bar_style = 'danger'
elif not os.path.exists(PYTHON_EXE):
    err_msg = f"Error: Python executable not found at {PYTHON_EXE}."
    with status_output:
        print(err_msg, file=sys.stderr)
    display(HTML(f"<p><font color='red'>{err_msg}</font></p>"))
    step_description.value = "Error: Python not found."
    overall_progress.bar_style = 'danger'
else:
    download_cmd = [PYTHON_EXE, script_path]
    with status_output:
        print(f"Using command: {' '.join(download_cmd)}\n")
    # Run the script from within the repository directory in case it uses relative paths
    success = run_command(download_cmd, "Download Models Script", "download_models.log", cwd=REPO_CLONE_PATH)

    if not success:
        step_description.value = "Error during Model Download Script. Check logs and output above."
        overall_progress.bar_style = 'danger'
    else:
        # Assuming success from run_command means it finished
        step_description.value = "Model Download Script Complete."
        overall_progress.value = 0.95 # Nearly done
        # Final success state
        step_description.value = "Setup Complete! Dependencies installed into /venv/main."
        overall_progress.value = 1.0
        overall_progress.bar_style = 'success'

## Section 4: Final Verification

Provides commands to verify the setup.

In [ ]:
import importlib

# Use the status_output widget for final verification prints
with status_output:
    print("--- Setup Notebook Finished ---")
    print(f"Workspace: {WORKSPACE_DIR}")
    print(f"Python Env: {PYTHON_ENV_PATH}")
    print(f"Notebook Logs: {NOTEBOOK_LOG_DIR}")

    print(f"\nVerify Packages in {PYTHON_ENV_PATH} (first ~15 lines of pip list):")
    # Use the specific pip from the venv
    verify_pip_cmd = [PIP_EXE, 'list']
    result = subprocess.run(verify_pip_cmd, capture_output=True, text=True, check=False, encoding='utf-8', errors='replace')
    if result.returncode == 0:
        output_lines = result.stdout.splitlines()
        print('\n'.join(output_lines[:15]))
    else:
        print(f"Error running pip list: {result.stderr}")

    print("\nVerify PyTorch with CUDA support:")
    # Use the full path to the environment's python to be absolutely sure
    verify_torch_cmd = [
        PYTHON_EXE, 
        '-c', 
        "import torch; print(f'PyTorch version: {torch.__version__}'); print(f'CUDA available: {torch.cuda.is_available()}'); print(f'Device count: {torch.cuda.device_count()}'); print(f'CUDA version: {torch.version.cuda}') if torch.cuda.is_available() else print('CUDA not found by PyTorch')"
    ]
    # Run and capture output to print within the widget
    result_torch = subprocess.run(verify_torch_cmd, capture_output=True, text=True, check=False, encoding='utf-8', errors='replace')
    print(result_torch.stdout)
    if result_torch.returncode != 0:
        print(f"Error verifying PyTorch: {result_torch.stderr}", file=sys.stderr)

    print(f"\nCheck Notebook Logs in: {NOTEBOOK_LOG_DIR}")
    # Use Python os module for listing directory
    try:
        log_files = os.listdir(NOTEBOOK_LOG_DIR)
        print("Log files:", log_files)
    except Exception as e:
        print(f"Error listing log directory: {e}")

    print(f"\nCheck VisoMaster clone in: {REPO_CLONE_PATH}")
    # Use Python os module for listing directory
    try:
        repo_files = os.listdir(REPO_CLONE_PATH)
        print("Repo files/dirs (partial):", repo_files[:10]) # Show first 10
    except Exception as e:
        print(f"Error listing repo directory: {e}")

    print("\nReminder: Check Supervisor/Portal logs via Instance Portal UI or terminal in /var/log/supervisor/ and /var/log/portal/")
    # Removed reference to entrypoint.log as we disabled that logging method
